# Feature Track 3: Query Improvement Methods

# Setup

In [2]:
from loguru import logger
import sys


from conversational_toolkit.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from conversational_toolkit.agents.base import QueryWithContext

from sme_kt_zh_collaboration_rag.feature0_baseline_rag import (
    load_chunks,
    build_vector_store,
    build_llm,
    build_agent,
    SYSTEM_PROMPT,
    EMBEDDING_MODEL,
)

logger.remove()
logger.add(sys.stderr, level="WARNING")  # hides DEBUG

c:\Users\sieverin\SDSC\Code\sme-kt-zh-collaboration-rag\rag_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


1

In [3]:
chunks = load_chunks(5)

embedding_model = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
vector_store = await build_vector_store(chunks, embedding_model, reset=False)
llm = build_llm("openai", model_name="gpt-4o-mini")

5


In [4]:
agent = build_agent(
    vector_store,
    embedding_model,
    llm,
    top_k=5,
    system_prompt=SYSTEM_PROMPT,
    number_query_expansion=0,
)

# Query Expansion

See conversational_toolkit.utils.retriever.py -> query_expansion and reciprocal_rank_fusion

In [5]:
logger.add(sys.stderr, level="DEBUG")

2

In [6]:
agent_query_expansion = build_agent(
    vector_store,
    embedding_model,
    llm,
    top_k=5,
    system_prompt=SYSTEM_PROMPT,
    number_query_expansion=5,
)

2026-02-25 16:57:17.390 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:build_agent:337 - RAG agent ready (top_k=5  query_expansion=5)


In [7]:
query = "Which pallets in our portfolio have a third-party verified EPD?"
query_with_context = QueryWithContext(query=query, history=[])

response = await agent.answer(query_with_context)

print("Answer:")
print(f"{response.content}")
print(f"Sources ({len(response.sources)}):")
for src in response.sources:
    source_file = src.metadata.get("source_file", "?")
    print(f"  {source_file!r}  |  {src.title!r}")

2026-02-25 16:57:17.424 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - sentence-transformers/all-MiniLM-L6-v2 embeddings size: (1, 384)


Answer:
Based on the provided excerpts, I can confirm that PrimePack AG has a portfolio of pallets, but the specific details regarding which pallets have third-party verified Environmental Product Declarations (EPDs) are not explicitly listed in the excerpts. 

The active pallet portfolio includes:
- Noé Pallet (32-100, CPR System)
- Wooden Pallet 1208 (32-101, CPR System)
- Recycled Plastic Pallet (32-102, CPR System)
- Logypal 1 (32-103, Relicyc)
- LogyLight (32-104, Relicyc)
- EP 08 (32-105, StabilPlastik) 

However, the excerpts do not specify which of these pallets have third-party verified EPDs. The information regarding EPD verification is mentioned, but not linked to specific products in the portfolio. 

Therefore, I do not know which pallets in your portfolio have a third-party verified EPD.
Sources (5):
  'EPD_pallet_relicyc_logypal1.pdf'  |  '### THIRD-PARTY VERIFICATION'
  'ART_response_inquiry_frische_felder.md'  |  '### Response to Section 3: General'
  'EVALUATION_qa_gro

In [8]:
query = "Which pallets in our portfolio have a third-party verified EPD?"
query_with_context = QueryWithContext(query=query, history=[])

response = await agent_query_expansion.answer(query_with_context)

print("Answer:")
print(f"{response.content}")
print(f"Sources ({len(response.sources)}):")
for src in response.sources:
    source_file = src.metadata.get("source_file", "?")
    print(f"  {source_file!r}  |  {src.title!r}")

2026-02-25 16:57:24.491 | DEBUG    | conversational_toolkit.llms.openai:generate:87 - Completion: ChatCompletion(id='chatcmpl-DDBYIJPetd8uVDuThOAEMk5F6GhOE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Which pallets in our inventory have third-party verified Environmental Product Declarations?\n2. How to find pallets with verified EPD in our product range?\n3. List of pallets with third-party EPD certification in our catalog.\n4. What are the benefits of third-party verified EPD for pallets?\n5. How to verify if a pallet has a third-party EPD?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1772035042, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_373a14eb6f', usage=CompletionUsage(completion_tokens=76, prompt_tokens=103, total_tokens=179, completion_tokens_details=CompletionTokensDetails(accepted_prediction

Answer:
Based on the provided excerpts, the only pallet mentioned with a third-party verified Environmental Product Declaration (EPD) is the **Logypal 1**. This pallet is classified as distribution packaging and is composed of over 99% polyolefins and other trace materials, with a reported recycled plastic content of 100% (as per the EPD reference certificate Accr. No. 021/2020 dated 26-03-2023) (source: excerpt from document id "b8133ef5-25d6-4a18-8e3a-638473bcc3af").

The third-party verification for the EPD was conducted by Bureau Veritas Italia S.p.A., an accredited certification body (source: excerpt from document id "9fe53c3c-32c8-4b4e-8f37-2ad9261fd30e"). 

If you need further details or have additional questions, feel free to ask!
Sources (5):
  'ART_response_inquiry_frische_felder.md'  |  '### Response to Section 3: General'
  'EPD_pallet_relicyc_logypal1.pdf'  |  '### THIRD-PARTY VERIFICATION'
  'EPD_pallet_relicyc_logypal1.pdf'  |  '# CONTENT DECLARATION'
  'EPD_tape_IPG_hot

# Hallucinate answer

In [9]:
agent_hyde = build_agent(
    vector_store,
    embedding_model,
    llm,
    top_k=5,
    system_prompt=SYSTEM_PROMPT,
    number_query_expansion=0,
    enable_hyde=True,
)

2026-02-25 16:57:30.053 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:build_agent:337 - RAG agent ready (top_k=5  query_expansion=0)


In [11]:
query = "Which pallets in our portfolio have a third-party verified EPD?"
query_with_context = QueryWithContext(query=query, history=[])

response = await agent.answer(query_with_context)

print("Answer:")
print(f"{response.content}")
print(f"Sources ({len(response.sources)}):")
for src in response.sources:
    source_file = src.metadata.get("source_file", "?")
    print(f"  {source_file!r}  |  {src.title!r}")

2026-02-25 16:58:02.476 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - sentence-transformers/all-MiniLM-L6-v2 embeddings size: (1, 384)


Answer:
Based on the provided excerpts, I can confirm that PrimePack AG has a portfolio of pallets, but the specific information regarding which pallets have third-party verified Environmental Product Declarations (EPDs) is not explicitly listed in the excerpts. 

The active pallet portfolio includes:
- Noé Pallet (32-100, CPR System)
- Wooden Pallet 1208 (32-101, CPR System)
- Recycled Plastic Pallet (32-102, CPR System)
- Logypal 1 (32-103, Relicyc)
- LogyLight (32-104, Relicyc)
- EP 08 (32-105, StabilPlastik) 

However, the excerpts do not specify which of these pallets have third-party verified EPDs. The information regarding EPD verification is mentioned, but not linked to specific products in the portfolio. 

Therefore, I do not know which pallets in your portfolio have a third-party verified EPD.
Sources (5):
  'EPD_pallet_relicyc_logypal1.pdf'  |  '### THIRD-PARTY VERIFICATION'
  'ART_response_inquiry_frische_felder.md'  |  '### Response to Section 3: General'
  'EVALUATION_qa_

In [10]:
query = "Which pallets in our portfolio have a third-party verified EPD?"
query_with_context = QueryWithContext(query=query, history=[])

response = await agent_hyde.answer(query_with_context)

print("Answer:")
print(f"{response.content}")
print(f"Sources ({len(response.sources)}):")
for src in response.sources:
    source_file = src.metadata.get("source_file", "?")
    print(f"  {source_file!r}  |  {src.title!r}")

2026-02-25 16:57:32.033 | DEBUG    | conversational_toolkit.llms.openai:generate:87 - Completion: ChatCompletion(id='chatcmpl-DDBYQnIPuMaZq53rlDjPYSilntpib', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To identify which pallets in your portfolio have a third-party verified Environmental Product Declaration (EPD), you would need to review the documentation for each pallet product. Look for EPD labels or certificates that indicate third-party verification, typically provided by recognized organizations. This information is often available in product specifications or sustainability reports from your suppliers.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1772035050, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_373a14eb6f', usage=CompletionUsage(completion_tokens=65, prompt_tokens=63, total_tokens=128, complet

Answer:
Based on the provided excerpts, I do not have specific information about which pallets in PrimePack AG's portfolio have a third-party verified Environmental Product Declaration (EPD). The excerpts mention that there is independent third-party verification of declarations and data according to ISO 14025:2006, but they do not specify which products or pallets have such EPDs.

The active pallet portfolio includes:
- Noé Pallet (32-100, CPR System)
- Wooden Pallet 1208 (32-101, CPR System)
- Recycled Plastic Pallet (32-102, CPR System)
- Logypal 1 (32-103, Relicyc)
- LogyLight (32-104, Relicyc)
- EP 08 (32-105, StabilPlastik)

However, the excerpts do not indicate whether any of these specific pallets have a third-party verified EPD. Therefore, I cannot provide a definitive answer regarding which pallets have a third-party verified EPD.
Sources (5):
  'EPD_pallet_relicyc_logypal1.pdf'  |  '### THIRD-PARTY VERIFICATION'
  'EPD_tape_IPG_hotmelt.pdf'  |  '## **EPD Programme Informatio

---------------------------